<a href="https://colab.research.google.com/github/wayanpdw/TextMining-IndonesianHateSpeech/blob/master/Text_Mining_Twitter_Sentiment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Text Mining - Twitter Sentiment**
I Wayan Pasek Dian Wahyudi | DS0114 | Telkom DTI 2020

# Pendahuluan
Text mining memiliki definisi penambangan data yang berupa teks dimana sumber data biasanya di dapatkan dari dokumen, dan tujuannya adalah mencari kata-kata yang dapat mewakili isi dari dokumen sehingga dapat dilakukan analisa keterhubungan antardokumen.

Kali ini akan dianalisa data text yang dikumpulkan dari sumber twitter.com

Dataset terdiri dari dua buah kolom yang masing-masing berisi kumpulan tweet dan pelabelan dari tweet yang bersangkutan.

Selanjutnya kumpulan tweet pada dataset akan dikategorikan dengan label "HS" (Hate-speech) dan "Non_HS" (Non-hate-speech).


#**Menyiapkan Data**


Mengimpor *Libraries*


In [113]:
import pandas as pd
import pandas as pd
import matplotlib.pyplot as plt
import csv
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

Mengimpor dataset

In [114]:
df = pd.read_csv('https://raw.githubusercontent.com/wayanpdw/TextMining-IndonesianHateSpeech/master/twitter_data_clean.csv' , sep=",")

# Memunculkan 5 data pertama dalam sebuah dataframe baru (opsional)
df.head()

,Tweet,Label
0,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,Non_HS
1,RT @baguscondromowo: Mereka terus melukai aksi...,Non_HS
2,Sylvi: bagaimana gurbernur melakukan kekerasan...,Non_HS
3,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",Non_HS
4,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,Non_HS


In [115]:
# Menampilkan 5 data terakhir
df.tail()

,Tweet,Label
708,Muka Si BABi Ahok Tuh Yg Mirip SERBET Lantai.....,HS
709,"Betul bang hancurkan merka bang, musnahkan chi...",HS
710,"Sapa Yg bilang Ahok anti korupsi!?, klo grombo...",HS
711,"Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar...",HS
712,Mudah2an gw ketemu sama SI BABI IWAN BOPENG DI...,HS


## **Preprocessing Data**

Preprocessing data dilakukan dengan melakukan penyeimbangan data karena dataset awal merupakan imbalanced data, dimana jumlah data berlabel hatespeech dan non hatespeech tidak seimbang. Teknik balanced data yang dilakukan menggunakan metode undersampling (memotong jumlah kelas yang berlimpah sehingga jumlahnya menjadi sama dengan kelas lainnya). Kenapa hal ini harus dilakukan? karena pada dasarnya data yang imbalanced merupakan masalah umum dalam machine learning. Jika dipaksakan melakukan pemodelan, maka model akan sia-sia walaupun memiliki akurasi yang tinggi.

In [116]:
# Cek apakah ada data yang kosong
print(df.isnull().values.any())

True


Ada data kosong

In [117]:
# Drop data yang kosong
df = df.dropna(axis=0, subset=['Tweet'])
print(df)

                                                 Tweet   Label
0    RT @spardaxyz: Fadli Zon Minta Mendagri Segera...  Non_HS
1    RT @baguscondromowo: Mereka terus melukai aksi...  Non_HS
2    Sylvi: bagaimana gurbernur melakukan kekerasan...  Non_HS
3    Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...  Non_HS
4    RT @lisdaulay28: Waspada KTP palsu.....kawal P...  Non_HS
..                                                 ...     ...
708  Muka Si BABi Ahok Tuh Yg Mirip SERBET Lantai.....      HS
709  Betul bang hancurkan merka bang, musnahkan chi...      HS
710  Sapa Yg bilang Ahok anti korupsi!?, klo grombo...      HS
711  Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar...      HS
712  Mudah2an gw ketemu sama SI BABI IWAN BOPENG DI...      HS

[712 rows x 2 columns]


In [118]:
# periksa kembali apa data kosong sudah di drop
print(df.isnull().values.any())

False


Data kosong berhasil di-drop

In [119]:
# Cek jumlah data
df['Label'].count()

712

In [120]:
#menghitung proporsi data label dengan jumlah yg tidak seimbang
df["Label"].value_counts() 

Non_HS    453
HS        259
Name: Label, dtype: int64

 **Data Cleaning**

 Dalam tahap ini data akan dibersihkan dari mentions, hasgtag, retweets, urls, case folding, tokenization, stopword dan stemming.

In [121]:
tweets = []
for tweet in df['Tweet'] :
    tweet = tweet.lower()
    tweet = re.sub(r"(?:\@|https?\://)\S+", "", tweet)
    tweet = re.sub(r"http\S+", "", tweet)
    tweet = re.sub('\n', '', tweet)
    tweet = re.sub('rt', '', tweet)
    tweet = re.sub("[^a-zA-Z^']", " ", tweet)
    tweet = re.sub(" {2,}", " ", tweet)
    tweet = tweet.strip()
    tweets.append(tweet)
df['clean_text'] = tweets

df.head()

,Tweet,Label,clean_text
0,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,Non_HS,fadli zon minta mendagri segera menonaktifkan ...
1,RT @baguscondromowo: Mereka terus melukai aksi...,Non_HS,mereka terus melukai aksi dalam rangka memenja...
2,Sylvi: bagaimana gurbernur melakukan kekerasan...,Non_HS,sylvi bagaimana gurbernur melakukan kekerasan ...
3,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",Non_HS,ahmad dhani tak puas debat pilkada masalah jal...
4,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,Non_HS,waspada ktp palsu kawal pilkada


**Stemming**

In [122]:
pip install Sastrawi

In [123]:
pip install nltk

In [124]:
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
factory = StemmerFactory()
stemmer = factory.create_stemmer()

def stem(tweet) :
    hasil = stemmer.stem(tweet)
    return hasil

df['clean_text'] = df.apply(lambda row : stem(row['clean_text']), axis = 1)

df.head()

,Tweet,Label,clean_text
0,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,Non_HS,fadli zon minta mendagri segera nonaktif ahok ...
1,RT @baguscondromowo: Mereka terus melukai aksi...,Non_HS,mereka terus luka aksi dalam rangka penjara ah...
2,Sylvi: bagaimana gurbernur melakukan kekerasan...,Non_HS,sylvi bagaimana gurbernur laku keras perempuan...
3,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",Non_HS,ahmad dhani tak puas debat pilkada masalah jal...
4,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,Non_HS,waspada ktp palsu kawal pilkada


**Stopwards Removal**

In [125]:
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory
# from nltk.tokenize import word_tokenize

factory = StopWordRemoverFactory()
stopword = factory.create_stop_word_remover()

def stopwords(tweet) :
    tweet = tweet.translate(str.maketrans('','',string.punctuation)).lower()
    return stopword.remove(tweet)

df['clean_text'] = df.apply(lambda row : stem(row['clean_text']), axis = 1)

df

,Tweet,Label,clean_text
0,RT @spardaxyz: Fadli Zon Minta Mendagri Segera...,Non_HS,fadli zon minta mendagri segera nonaktif ahok ...
1,RT @baguscondromowo: Mereka terus melukai aksi...,Non_HS,mereka terus luka aksi dalam rangka penjara ah...
2,Sylvi: bagaimana gurbernur melakukan kekerasan...,Non_HS,sylvi bagaimana gurbernur laku keras perempuan...
3,"Ahmad Dhani Tak Puas Debat Pilkada, Masalah Ja...",Non_HS,ahmad dhani tak puas debat pilkada masalah jal...
4,RT @lisdaulay28: Waspada KTP palsu.....kawal P...,Non_HS,waspada ktp palsu kawal pilkada
...,...,...,...
708,Muka Si BABi Ahok Tuh Yg Mirip SERBET Lantai.....,HS,muka si babi ahok tuh yg mirip serbet lantai
709,"Betul bang hancurkan merka bang, musnahkan chi...",HS,betul bang hancur merka bang musnah china babi...
710,"Sapa Yg bilang Ahok anti korupsi!?, klo grombo...",HS,sapa yg bilang ahok anti korupsi klo grombolan...
711,"Gw juga ngimpi SENTILIN BIJI BABI AHOK, pcetar...",HS,gw juga ngimpi sentilin biji babi ahok pcetar ...


In [126]:
df.shape

(712, 3)

In [127]:
df['Label'].value_counts()

Non_HS    453
HS        259
Name: Label, dtype: int64

# **Membuat Model**

**Vektorisasi**

In [128]:
X = df['clean_text']
count_vect = CountVectorizer()
X_train_counts = count_vect.fit_transform(X)
X_train_counts.shape

(712, 2299)

In [129]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape

(712, 2299)

In [130]:
label = LabelEncoder()
y = label.fit_transform(df['Label'])


X_train, X_test, y_train, y_test = train_test_split(X_train_tfidf, y, test_size=0.2, random_state=0)

**Random Forest Classifier**

In [131]:
rf = RandomForestClassifier(max_depth=10, random_state=1)
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy : ",accuracy_score(y_test,y_pred))
print("\n Confusion Matrix : \n", confusion_matrix(y_test, y_preds))

              precision    recall  f1-score   support

           0       1.00      0.53      0.69        57
           1       0.76      1.00      0.86        86

    accuracy                           0.81       143
   macro avg       0.88      0.76      0.78       143
weighted avg       0.86      0.81      0.79       143

Accuracy :  0.8111888111888111

 Confusion Matrix : 
 [[30 27]
 [ 0 86]]


**K-Nearest Neighbours Classsifier**

In [132]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train,y_train)

y_pred = knn.predict(X_test)

print(classification_report(y_test, y_pred))
print("Accuracy : ",accuracy_score(y_test,y_pred))
print("\n Confusion Matrix : \n", confusion_matrix(y_test, y_preds))

              precision    recall  f1-score   support

           0       0.78      0.75      0.77        57
           1       0.84      0.86      0.85        86

    accuracy                           0.82       143
   macro avg       0.81      0.81      0.81       143
weighted avg       0.82      0.82      0.82       143

Accuracy :  0.8181818181818182

 Confusion Matrix : 
 [[30 27]
 [ 0 86]]
